# Instacart Advanced Analytics
# This notebook performs advanced analytics on the Instacart dataset.


In [0]:
%sql
-- 1. Product Purchase Patterns
-- Products most frequently bought together
WITH order_pairs AS (
  SELECT a.order_id, a.product_id as product_1, b.product_id as product_2
  FROM all_order_products a
  JOIN all_order_products b 
    ON a.order_id = b.order_id AND a.product_id < b.product_id
)

SELECT 
  p1.product_name as product_1_name,
  p2.product_name as product_2_name,
  COUNT(*) as pair_count
FROM order_pairs op
JOIN instacart.products p1 ON op.product_1 = p1.product_id
JOIN instacart.products p2 ON op.product_2 = p2.product_id
GROUP BY product_1_name, product_2_name
ORDER BY pair_count DESC
LIMIT 20;


product_1_name,product_2_name,pair_count
Bag of Organic Bananas,Organic Hass Avocado,64761
Bag of Organic Bananas,Organic Strawberries,64702
Organic Strawberries,Banana,58330
Banana,Organic Avocado,55611
Organic Baby Spinach,Banana,53395
Bag of Organic Bananas,Organic Baby Spinach,52608
Strawberries,Banana,43180
Banana,Large Lemon,43038
Organic Strawberries,Organic Hass Avocado,42333
Bag of Organic Bananas,Organic Raspberries,42283


In [0]:
# 2. User Segmentation Analysis

from pyspark.sql.functions import col, count, sum, avg
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans

# Create user features for segmentation
user_features = spark.sql("""
  SELECT 
    o.user_id,
    COUNT(DISTINCT o.order_id) as total_orders,
    AVG(o.days_since_prior_order) as avg_days_between_orders,
    AVG(o.order_hour_of_day) as avg_order_hour,
    COUNT(DISTINCT op.product_id) / COUNT(DISTINCT o.order_id) as avg_basket_size,
    SUM(CASE WHEN op.reordered = 1 THEN 1 ELSE 0 END) / COUNT(*) as reorder_ratio
  FROM instacart.orders o
  JOIN all_order_products op ON o.order_id = op.order_id
  GROUP BY o.user_id
""")

# Prepare data for KMeans
assembler = VectorAssembler(
    inputCols=["total_orders", "avg_days_between_orders", "avg_order_hour", "avg_basket_size", "reorder_ratio"],
    outputCol="features"
)
user_features_vector = assembler.transform(user_features)

# Fit KMeans model
kmeans = KMeans(k=4, seed=42)
model = kmeans.fit(user_features_vector)

# Add cluster predictions
user_segments = model.transform(user_features_vector)

# Show cluster centers
centers = model.clusterCenters()
print("Cluster Centers:")
for i, center in enumerate(centers):
    print(f"Cluster {i}: {center}")


Cluster Centers:
Cluster 0: [ 7.3105058 15.38016306 13.63885268 6.33294954 0.3501217 ]
Cluster 1: [45.01083535 7.63242912 13.2587431 3.11227042 0.69076125]
Cluster 2: [82.92923923 4.38239277 13.12513175 2.25966768 0.75087569]
Cluster 3: [21.56969448 11.44010328 13.45428359 4.16085056 0.56826807]

In [0]:
# Analyze user segments
user_segments.groupBy("prediction") \
    .agg(
        count("user_id").alias("user_count"),
        avg("total_orders").alias("avg_orders"),
        avg("avg_days_between_orders").alias("avg_days_between_orders"),
        avg("avg_order_hour").alias("avg_order_hour"),
        avg("avg_basket_size").alias("avg_basket_size"),
        avg("reorder_ratio").alias("avg_reorder_ratio")
    ) \
    .orderBy("prediction") \
    .show()


+----------+----------+------------------+-----------------------+------------------+------------------+-------------------+
prediction|user_count| avg_orders|avg_days_between_orders| avg_order_hour| avg_basket_size| avg_reorder_ratio|
+----------+----------+------------------+-----------------------+------------------+------------------+-------------------+
 0| 132567| 7.308613757571643| 15.380731752380413|13.638829583059872|6.3334814021822865|0.35007560316845027|
 1| 19284| 44.86392864550923| 7.644871681078375|13.259671808988767|3.1166985736102286| 0.6903390774398986|
 2| 5584| 82.49194126074498| 4.41613603172908| 13.12532069113389|2.2660340420054266| 0.7505497042407532|
 3| 48774|21.556792553409604| 11.443777123053417|13.454575443501227| 4.161471505426571| 0.5681972336662914|
+----------+----------+------------------+-----------------------+------------------+------------------+-------------------+

In [0]:
%sql
-- 3. Department and Aisle Analysis
-- Popular departments and aisles by time of day
SELECT 
  d.department,
  a.aisle,
  CASE 
    WHEN o.order_hour_of_day BETWEEN 5 AND 11 THEN 'Morning'
    WHEN o.order_hour_of_day BETWEEN 12 AND 16 THEN 'Afternoon'
    WHEN o.order_hour_of_day BETWEEN 17 AND 21 THEN 'Evening'
    ELSE 'Night'
  END as time_of_day,
  COUNT(*) as order_count
FROM all_order_products op
JOIN instacart.orders o ON op.order_id = o.order_id
JOIN instacart.products p ON op.product_id = p.product_id
JOIN instacart.aisles a ON p.aisle_id = a.aisle_id
JOIN instacart.departments d ON p.department_id = d.department_id
GROUP BY d.department, a.aisle, time_of_day
ORDER BY order_count DESC;


department,aisle,time_of_day,order_count
produce,fresh fruits,Afternoon,1508846
produce,fresh vegetables,Afternoon,1482001
produce,fresh fruits,Morning,1313031
produce,fresh vegetables,Morning,1164247
produce,fresh fruits,Evening,784450
produce,packaged vegetables fruits,Afternoon,748187
produce,fresh vegetables,Evening,736228
produce,packaged vegetables fruits,Morning,615034
dairy eggs,yogurt,Afternoon,587757
dairy eggs,yogurt,Morning,536288
